In [1]:
import struct # module for unpacking binary files
import os # module for using OS tools to navigate directories
import datetime as dt # module for converting time stamps to/from strings
import pyodbc # Module for ODBC for python to connect to database

def readTABin(fileName):
    with open(fileName,mode='rb') as f:

        # unpackP specifies the unpack parameters as 'name', type size (bytes), type char
        unpackP = [['desc',1,'s'],
                   ['avgs',4,'l'],
                   ['gPos',8,'d'],
                   ['delays',8,'d']]; 
        
        # define expSet as a dictionary, this will be returned to the caller
        expSet = {}

        for packRule in unpackP:
            # LabVIEW encodes binary data as header-data. 
            # The header is type unsigned long and contains the data size in bytes
            b = f.read(4) # long is 4 bytes
            h = struct.unpack('>L',b)[0] # upack method returns type tuple, take 1st element
            b = f.read(h*packRule[1]) # The char array is h bytes * type size
            expSet[packRule[0]] = (struct.unpack('>'+str(h)+packRule[2],b))

        # format expSet for easier reading
        expSet['desc'] = expSet['desc'][0].decode('utf-8')
        expSet['avgs'] = {'frames':expSet['avgs'][0],
                          'gPos':expSet['avgs'][1],
                          'rpts':expSet['avgs'][2]}
        
    return expSet


In [2]:
# Test readTaBin function
# Return the current working directory, which will be the script dir when run in jupyter notebook
parentDir = os.path.abspath(''); 
fileName = os.path.join(parentDir,'SampleData','19-12-05_14h53m12s_0p04Fl_0V_pH13_100mM_460nm_cont_0-2ns_spol');

expSet = readTABin(fileName)
print(expSet['desc'])

Sample L, Fluence dependence measurements at pH13 and 100mM Na+ ionic concentration


In [2]:
# Extract Metadata from TA bin files inside a given directory
filePath = 'C:\\Users\\User\\Documents\\UserData Backup\\TA data\\Ilya'

# uses os.walk to recursively search dirs in filePath, excludes files in paths that contains case-insensitive 'bad' and files that contain extensions
files = [[fiDir,fi] for fiDir,_,fiList in os.walk(filePath) for fi in fiList if not ('bad' in fiDir or 'Bad' in fiDir) and '.' not in fi]

# retrieve all descriptions in the file list
descs = [readTABin(os.path.join(fi[0],fi[1]))['desc'] for fi in files]

# Extract timestring from files into a list
tsStrs = [fi[1][0:18] for fi in files]

# Format timestring into a datetime object list
tsDT = [dt.datetime.strptime(ts,'%y-%m-%d_%Hh%Mm%Ss') for ts in tsStrs]

# Format the datetime object list into a SQL string
tsStrs = [str(ts) for ts in tsDT]

In [3]:
# Insert legacy TR bin data into Raman Database

# construct a connection object with DSN Raman Access and
conn = pyodbc.connect('DSN=Raman Access')

# Create a cursor to execute SQL INSERT
crsr = conn.cursor()

# Set autocommit to false so that database is updated only if query executes without error
conn.autocommit = False

# try to execute query
try:
    # Combine different lists and loop over their elements
    for (fi, des, ts) in zip(files, descs, tsStrs):
        # Execute parameterized query
        crsr = crsr.execute('INSERT INTO TRAcquisition '
                            '(FilePath, FileName, Description, SoftwareVersionID, DatabaseVersionID, [TimeStamp])' 
                            'VALUES (?, ?, ?, ?, ?, ?);', fi[0], fi[1], des, 2, 2, ts)
except pyodbc.DatabaseError as err:
    # Undo any damage and return the exception to the user
    conn.rollback()
    print(err)
else:
    # commit record updates to database
    conn.commit()
    
# Note: Close connection by shutting down notebook